# Validation Part B Responses

1. 
    1. The best possible score from an $R^2$ model is 1, indicating the model perfectly models the data. With real world data, getting such a value is almost impossible (unless your dataset has only 2 points).
    1. Simply returning the average as the model prediction would yield an $R^2$ value of 0.
    1. It is possible to get arbitrarily low negative scores by simply making the model more and more incorrrect with its predictions.
1. 
   1. The best possible accuracy a model can get is 100%, indicating all items were properly classified.
   1. If you had a model that only predicted class A, then in a dataset with $X\%$ items of type class A, the model would still get an accuracy of $X\%$. In the specific case of the question, the model would get an 85% accuracy; this is obviously undesireable, as the model clearly is not doing anything to actually classify data.
   1. On any data set, it is always possible to get an accuracy of 0% by perfectly mislabeling every item (which is impresive in its own right).
1. A recall score of 0 for class A indicates that there were no true positives for class A. Thus, the model never classifies an item from class A as belonging to class A; instead the model will always return either class B or C (though we can't know more than that).
1. A precision score of 1 for class A indicates that there were no false positives for class A. However, this tells us nothing about the true positives for class A, so knowing the precision score alone tells us nothing about what the model would do for something actually belonging to class A.
1. Just from knowing the AUC is 1, we cannot actually say anything about what the model's probabilty output would be. We do know the model would predict the item belongs to class A, however we cannot say with what probability it would do so. Indeed, in the case of AUC, all we really know from this is that the model will always assign a higher probability of belonging to class A to items actually from class A than from class B.
1. As above, just from knowing the AUC is 0, we cannot actually say anything about what the model's probabilty output would be. We do know the model would predict the item belongs to class B, however we cannot say with what probability it would do so. Indeed, in the case of AUC, all we really know from this is that the model will never assign a higher probability of belonging to class A to items actually from class A than from class B.